In [ ]:
#!pip install --upgrade pip
#!pip install numpy
#!pip install pandas
#!pip install xlrd

In [ ]:
import numpy as np
import pandas
from IPython.display import display, HTML
pandas.set_option('display.max_colwidth', -1)

print('Reading metadata...')
metadata_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/variables_acs2016_clara.csv', low_memory=False, index_col=0, encoding='utf-8')
geo_metadata_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/geo_variables_acs2016_clara.csv', low_memory=False, index_col=0, encoding='utf-8')

print('Reading geography...')
geo_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/geo_acs2016_clara.csv', low_memory=False, index_col=0, encoding='utf-8')

print('Reading data (estimates)...')
data_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/data_acs2016_clara.csv', low_memory=False, index_col=0, encoding='utf-8')

print('Reading margins of error...')
error_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/error_acs2016_clara.csv', low_memory=False, index_col=0, encoding='utf-8')

print('Calculating geography sumary levels...')
data_df['SUMLEVEL'] = data_df['GEOID'].apply(lambda x: int(x[0:3]))

print('Documenting geography summary levels...')
sumlevel_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/geography/geo_summary_levels.csv', encoding='utf-8')
sumlevel_df = sumlevel_df.set_index('SUMLEVEL', drop=False)

print('Documenting state codes')
geo_df[['STATE','STUSAB']].dropna().drop_duplicates().sort_values('STATE').to_csv('/home/idies/workspace/raddick_acs_data/geography/statecodes.csv', encoding='utf-8', index=False)
statecodes_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/geography/statecodes.csv', encoding='utf-8')
statecodes_df['STATE'] = statecodes_df['STATE'].astype('int')
statecodes_df = statecodes_df.set_index('STATE', drop=False)

print('\n')
print('Done!')

In [ ]:
#metadata_df
#metadata_df[metadata_df['sequence_number'] == 150]
#metadata_df
#B26001_001
#GROUP QUARTERS POPULATION for Population In Group Quarters% Total

#string1 = 'Baltimore'
#string2 = ''
#string3 = ''

#geovarlist = metadata_df['variable'][metadata_df['description'].apply(lambda x: (string1 in x) & (string2 in x) & (string3 in x))].values.tolist()
#geo_df[geo_df['NAME'].apply(lambda x: (string1 in x))]

#print('{0:}'.format(geo_df['NAME'].loc['05000US24510']))
#print('{0:} {1:,.0f}'.format(metadata_df['description'].loc['B26001_001'], data_df['B26001_001'].loc['05000US24510']))

#data_df['B26001_001'].loc['05000US24510']
#data_df      
#05000US24510
#data_df['B26001_001'].sample(1)

geolist = geo_df['GEOID'].sample(3).values.tolist()
#geolist.append('afwwfe')
    
thelist = ['NAME','B01001_001', 'B26001_001']
#thelist.append('xxx')
#thelist  = []
#ourdata_df = get_data(data_df, thelist, geolist)
#ourdata_df

phrases = []
phrases.append('Total Population')
phrases.append('Female')
#phrases.append('AGE')
#phrases.append('Under 5')
#phrases.append('Male')

thegeolist = find_geography(data_df, 9)
#thesevars = find_variables4(metadata_df, phrases)
#find_variables(metadata_df, phrases)
#metadata_df[metadata_df['variable'].apply(lambda x: x in thesevars)]
#print(thesevars)
#thegeo.groupby('SUMLEVEL').size()
ourdata_df = get_data(data_df, thelist, thegeolist)
ourdata_df
#print(thegeolist)
#sumlevel_df['description']

In [ ]:
def find_geography(ddf, wantlevel):
    gdf = []
    gdf = ddf['GEOID'][ddf['SUMLEVEL'] == wantlevel].values.tolist()
    if (len(ddf['GEOID'][ddf['SUMLEVEL'] == wantlevel]) == 0):
        showme = '<p><b>WARNING:</b> specified geography level ({0:.0f}) does not exist.<br />'.format(wantlevel)
        showme += 'The table below shows possible values.</p>'
        display(HTML(showme))
        
        helpshow_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/geography/geo_summary_levels.csv')
        helpshow_df = helpshow_df.set_index('SUMLEVEL')
        #print(helpshow_df['description'])
        showme = '<table><tr><th>SUMLEVEL</th><th>Description</th></tr>'
        for slvl, row in helpshow_df.iterrows():
#            print(str(slvl))
            showme += '<tr><td>'+str(slvl)+'</td><td>'+str(row['description'])+'</td></tr>'
        showme += '</table>'
#        print(showme)
        display(HTML(showme))
    else:
        gdf = ddf['GEOID'][ddf['SUMLEVEL'] == wantlevel].values.tolist()
    return gdf
    
def find_variables4(mdf, tofindlist):
    # NEED TO REWRITE TO USE REGEX INSTEAD, because now only "Male" returns men; "male" returns women b/c feMALE
    varlist = []
    if (len(tofindlist) == 0):
        print('CAUTION: No search phrases specified, returning every variable!')
    if (len(tofindlist) > 4):
        print('ERROR: list contains more than four search phrases, searching for only the first three')
    try:
        string1 = tofindlist[0]
    except IndexError:
        string1 = ''
    try:
        string2 = tofindlist[1]
    except IndexError:
        string2 = ''
    try:
        string3 = tofindlist[2]
    except IndexError:
        string3 = ''
    try:
        string4 = tofindlist[3]
    except IndexError:
        string4 = ''
            
    varlist = mdf['variable'][mdf['description'].apply(lambda x: (string1 in x) & (string2 in x) & (string3 in x) & (string4 in x))].values.tolist()
    return varlist

def get_data(df, varlist, geolist):
    y_df = get_geography(df, geolist)
    rdf = get_columns(y_df, varlist)
    return rdf

def get_columns(df, varlist):
    if (len(varlist) == 0):
        print('CAUTION: No variables specified')
    rdf = pandas.DataFrame()
    if (len(df) > 0):
        vars_that_exist = ['GEOID']
        for thisvar in varlist:
            if (thisvar in df.columns):
                vars_that_exist.append(thisvar)
            else:
                print('Variable {0:} not found. Skipping.'.format(thisvar))

        rdf = df[vars_that_exist]
        rdf.index.name = ''
    else:
        print('CAUTION: No geopgraphies specified')
    return rdf

def get_geography(df, geolist):
    rdf = pandas.DataFrame()
    if (len(df) > 0):
        for thisgeo in geolist:
            try:
                rdf = rdf.append(df.loc[thisgeo])
            except:
                print('Geography {0:} not found. Skipping.'.format(thisgeo))
    else:
        print('No variables specified')
    return rdf
    
print('Done')

In [ ]:
sumlevel_df

In [ ]:
#geo_df[geo_df['SUMLEVEL'] != 40].groupby('STATE').size()
# US = 1 if this is data for the U.S., only for SUMLEVEL = 10
# REGION from 1 to 4, only for SUMLEVEL = 20
# DIVISION from 1 to 9, only for SUMLEVEL = 30
# STATE from 1 (AL) to 72 (PR), but skips several values. But they map one-to-one with STUSAB, so we don't really need them.
# COUNTY from 1 to 810, different for different states. Numbers mostly alphabetical list, but with skips (old names?)

# PLACE has only 596 distinct values, they seem mostly random
# CBSA is probably combined statistical area, seems mostly random

#geo_df[['STATE','STUSAB','NAME']][geo_df['STATE'] == 3].sample(10)
#geo_df.groupby('PLACE').size()
geo_df[['SUMLEVEL','STUSAB','PLACE','NAME']][geo_df['SUMLEVEL'] == 170]

In [ ]:
data_df['SUMLEVEL'] = data_df['GEOID'].apply(lambda x: int(x[0:3]))

print('Estimated U.S. population: {0:,.0f}'.format(data_df['B01001_001'].loc['01000US']))

print('For the four census regions: {0:,.0f}'.format(data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:7]) == '02000US'].sum()))
print('\n')
print('Population by census region:')
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US1'], data_df['B01001_001'].loc['02000US1']))
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US2'], data_df['B01001_001'].loc['02000US2']))
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US3'], data_df['B01001_001'].loc['02000US3']))
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US4'], data_df['B01001_001'].loc['02000US4']))

#print('Men: {0:,.0f} (error: {1:,.0f})'.format(data_df['B01001_002'].loc['01000US'], error_df['B01001_002'].loc['01000US']))
#print('Women: {0:,.0f} (error: {1:,.0f})'.format(data_df['B01001_026'].loc['01000US'], error_df['B01001_026'].loc['01000US']))

#print('Margin of error: {0:,.0f}'.format(error_df['B01001_001'].loc['01000US']))

In [ ]:
#data_df[data_df['SUMLEVEL'] == 40]

display(HTML('<h2>Population by state</h2>'))

statetable = '<table><tr><th>State</th><th>Population</th></tr>'
for thisgeoid, thisrow in data_df[data_df['GEOID'].apply(lambda x: x[0:5]) == '04000'].iterrows():
    statetable += '<tr>'
    statetable += '<td>{0:}</td><td style="text-align:right;padding:5px">{1:,.0f}</td>'.format(thisrow['NAME'], thisrow['B01001_001'])
    statetable += '</tr>'
statetable += '</table>'

display(HTML(statetable))

total_population = data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:5]) == '04000'].sum()

display(HTML(('TOTAL of all 50 states: <b>{0:,.0f}</b> (not equal to national b/c military etc. is federal)'.format(total_population))))
#print('For the four census regions: {0:,.0f}'.format(data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:7]) == '02000US'].sum()))
#data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:7]) == '02000US']



In [ ]:
#geo_df[['SUMLEVEL','STUSAB','COUNTY','NAME']][(geo_df['SUMLEVEL'] == 310) & (geo_df['STUSAB'] == 'MD')].sort_values('COUNTY')
#geo_df[geo_df['SUMLEVEL'] == 310].groupby('CBSA').size().sort_values()
geo_df.sample(1).T